 # 第 8 章 数据规整:聚合、合并和重塑

## 8.1 层次化索引

In [3]:
import numpy as np
import pandas as pd

In [ ]:
#层次化索引(hierarchical indexing)是 pandas 的一项重要功能，它使你能 在一个轴上拥有多个(两个以上)索引级别。
#抽象点说，它使你能以低维度形 式处理高维度数据。

In [4]:
#创建一个 Series，并用一个 由列表或数组组成的列表作为索引:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.733505
   2    1.910682
   3   -1.572898
b  1    0.155768
   3   -0.372890
c  1    0.521874
   2    1.042330
d  2   -0.070542
   3   -0.008924
dtype: float64

In [5]:
#看到的结果是经过美化的带有 MultiIndex 索引的 Series 的格式。索引之间的 “间隔”表示“直接使用上面的标签”:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [6]:
#对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集的操作更简单:
data['b']

1    0.155768
3   -0.372890
dtype: float64

In [7]:
data['b':'c']

b  1    0.155768
   3   -0.372890
c  1    0.521874
   2    1.042330
dtype: float64

In [8]:
data.loc[['b', 'd']]

b  1    0.155768
   3   -0.372890
d  2   -0.070542
   3   -0.008924
dtype: float64

In [9]:
#有时甚至还可以在“内层”中进行选取:
data.loc[:, 2]

a    1.910682
c    1.042330
d   -0.070542
dtype: float64

In [10]:
#层次化索引在数据重塑和基于分组的操作(如透视表生成)中扮演着重要的角 色。
#可以通过 unstack 方法将这段数据重新安排到一个 DataFrame 中:
data.unstack()

,1,2,3
a,0.733505,1.910682,-1.572898
b,0.155768,NaN,-0.372890
c,0.521874,1.042330,NaN
d,NaN,-0.070542,-0.008924


In [11]:
#unstack 的逆运算是 stack:
data.unstack().stack()

a  1    0.733505
   2    1.910682
   3   -1.572898
b  1    0.155768
   3   -0.372890
c  1    0.521874
   2    1.042330
d  2   -0.070542
   3   -0.008924
dtype: float64

In [13]:
#对于一个 DataFrame，每条轴都可以有分层索引:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [14]:
#各层都可以有名字(可以是字符串，也可以是别的 Python 对象)。如果指定了 名称，它们就会显示在控制台输出中:
frame.index.names = ['key1', 'key2']

In [15]:
frame.columns.names = ['state', 'color']

In [16]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
#有了部分列索引，因此可以轻松选取列分组
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [ ]:
#可以单独创建 MultiIndex 然后复用。上面那个 DataFrame 中的(带有分级名 称)列可以这样创建:
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green',
'Red', 'Green']],
                       names=['state', 'color'])


## 重排与分级排序

In [ ]:
#你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据 进行排序。
#swaplevel 接受两个级别编号或名称，并返回一个互换了级别的新 对象(但数据不会发生变化):

In [19]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [20]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [21]:
#而 sort_index 则根据单个级别中的值对数据进行排序。
#交换级别时，常常也会 用到 sort_index，这样最终结果就是按照指定顺序进行字母排序了:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [29]:
#先交换，后排序
frame.swaplevel(0, 1)   

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [30]:
frame.swaplevel(0, 1).sort_index(level=0)   

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 根据级别汇总统计

In [36]:
#许多对 DataFrame 和 Series 的描述和汇总统计都有一个 level 选项，它用于指 定在某条轴上求和的级别。
#我们可以根据行或 列上的级别来进行求和:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [37]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [39]:
frame.sum(level='color', axis=1)   （axis在此处作何解释 todo）

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## 使用 DataFrame 的列进行索引

In [40]:
#人们经常想要将 DataFrame 的一个或多个列当做行索引来用，或者可能希望将 行索引变成 DataFrame 的列。
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame                                   

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [41]:
#DataFrame 的 set_index 函数会将其一个或多个列转换为行索引，并创建一个 新的 DataFrame:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [42]:
#默认情况下，那些列会从 DataFrame 中移除，但也可以将其保留下来:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [ ]:
#reset_index 的功能跟 set_index 刚好相反，层次化索引的级别会被转移到列 里面:

In [43]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [44]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 合并数据集

pandas 对象中的数据可以通过一些方式进行合并:
1.pandas.merge 可根据一个或多个键将不同 DataFrame 中的行连接起来。SQL 或其他 关系型数据库的用户对此应该会比较熟悉，因为它实现的就是数据库的 join 操作。
2.pandas.concat 可以沿着一条轴将多个对象堆叠到一起。
3.实例方法 combine_first 可以将重复数据编接在一起，用一个对象中的值填充另一个
对象中的缺失值。

## 数据库风格的 DataFrame 合并

In [46]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a',
'b'],
                    'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [47]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [48]:
#这是一种多对一的合并。
#df1 中的数据有多个被标记为 a 和 b 的行，而 df2 中 key 列的每个值则仅对应一行。对这些对象调用 merge 即可得到:
pd.merge(df1, df2)  

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [49]:
#注意，我并没有指明要用哪个列进行连接。如果没有指定，merge 就会将重叠 列的列名当做键。不过，最好明确指定一下:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [50]:
#如果两个对象的列名不同，也可以分别进行指定:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a',
'b'],
                    'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [52]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [53]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [54]:
#可能你已经注意到了，结果里面 c 和 d 以及与之相关的数据消失了。
#默认情况 下，merge 做的是“内连接”;结果中的键是交集。其他方式还有"left"、 "right"以及"outer"。
#外连接求取的是键的并集，组合了左连接和右连接的效 果:
pd.merge(df1, df2, how='outer')   #使用两个表所有的键

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [55]:
#多对多的合并有些不直观。
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [56]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [57]:
pd.merge(df1, df2, on='key', how='left')    #使用左表都有的键，以left的df1为准

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [58]:
#多对多连接产生的是行的笛卡尔积。
#由于左边的 DataFrame 有 3 个"b"行，右边 的有 2 个，所以最终结果中就有 6 个"b"行。
#连接方式只影响出现在结果中的不 同的键的值:
pd.merge(df1, df2, how='inner')     #使用两个表都有的键

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [59]:
#要根据多个键进行合并，传入一个由列名组成的列表即可:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [60]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [61]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')  #使用两个表所有的键

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [64]:
#虽然你可以手工 处理列名重叠的问题(查看前面介绍的重命名轴标签)，
#但 merge 有一个更实 用的 suffixes 选项，用于指定附加到左右两个 DataFrame 对象的重叠列名上的 字符串:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [65]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [63]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [66]:
pd.merge(left, right, on='key1', suffixes=('_left',
'_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## 索引上的合并

In [67]:
#DataFrame 中的连接键位于其索引中。
#在这种情况下，你可以传入 left_index=True 或 right_index=True(或两个都传)以说明索引应该被用作 连接键:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [69]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a',
'b'])
right1

,group_val
a,3.5
b,7.0


In [70]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [71]:
#由于默认的 merge 方法是求取连接键的交集，因此你可以通过外连接的方式得 到它们的并集:
pd.merge(left1, right1, left_on='key', right_index=True,
how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [74]:
#对于层次化索引的数据，事情就有点复杂了，因为索引的合并默认是多键合并:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada','Nevada'],
                               'key2': [2000, 2001, 2002, 2001, 2002],
                               'data': np.arange(5.)})
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [75]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [76]:
#这种情况下，你必须以列表的形式指明用作合并键的多个列(注意用 how='outer'对重复索引值的处理):
pd.merge(lefth, righth, left_on=['key1', 'key2'],
right_index=True)    #默认是取交集

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [77]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [78]:
#同时使用合并双方的索引也没问题:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [79]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13,
14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [80]:
# 方法1:
pd.merge(left2, right2, how='outer', left_index=True,
right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [81]:
#DataFrame 还有一个便捷的 join 实例方法，它能更为方便地实现按索引合并。 
#它还可用于合并多个带有相同或相似索引的 DataFrame 对象，但要求没有重叠 的列。在上面那个例子中，我们可以编写:
# 方法2:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [84]:
#因为一些历史版本的遗留原因，DataFrame 的 join 方法默认使用的是左连接， 保留左边表的行索引。
#它还支持在调用的 DataFrame 的列上，连接传递的 DataFrame 索引:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [85]:
right1

,group_val
a,3.5
b,7.0


In [83]:
left1.join(right1, on='key')   #保留左边表的行索引

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [86]:
#对于简单的索引合并，你还可以向 join 传入一组 DataFrame，下一节会 介绍更为通用的 concat 函数，也能实现此功能:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.],
[16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York',
'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [87]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [88]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [89]:
left2.join([right2, another])    ##保留左边表的行索引

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [90]:
left2.join([right2, another], how='outer')    ##保留所有表的行索引

/Users/xl/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


## 轴向连接

In [94]:
#另一种数据合并运算也被称作连接(concatenation)、绑定(binding)或堆 叠(stacking)。
#NumPy 的 concatenation 函数可以用 NumPy 数组来做:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [97]:
np.concatenate([arr, arr], axis=1)   #axis=1代表行

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [98]:
#pandas 的 concat 函数提供了一种能够解决这些问题的可靠方式。
#我将给出一 些例子来讲解其使用方式。假设有三个没有重叠索引的 Series:

In [99]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s1

a    0
b    1
dtype: int64

In [100]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s2

c    2
d    3
e    4
dtype: int64

In [101]:
s3 = pd.Series([5, 6], index=['f', 'g'])
s3

f    5
g    6
dtype: int64

In [106]:
#对这些对象调用 concat 可以将值和索引粘合在一起:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [108]:
#默认情况下，concat 是在 axis=0 上工作的，最终产生一个新的 Series。
#如果 传入 axis=1，则结果就会变成一个 DataFrame(axis=1 是列):
pd.concat([s1, s2, s3], axis=1, sort=False)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [109]:
#这种情况下，另外的轴上没有重叠，从索引的有序并集(外连接)上就可以看 出来。传入 join='inner'即可得到它们的交集:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [111]:
pd.concat([s1, s4], axis=1, sort=False)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [112]:
pd.concat([s1, s4], axis=1, join='inner')  #f 和 g 标签消失了，是因为使用的是 join='inner'选项。

,0,1
a,0,0
b,1,1


In [113]:
#你可以通过 join_axes 指定要在其它轴上使用的索引:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b',
'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [114]:
#参与连接的片段在结果中区分不开。假设你想要在连接轴上创 建一个层次化索引。使用 keys 参数即可达到这个目的:
result = pd.concat([s1, s1, s3], keys=['one','two',
'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [115]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [116]:
#如果沿着 axis=1 对 Series 进行合并，则 keys 就会成为 DataFrame 的列头:
pd.concat([s1, s2, s3], axis=1, keys=['one','two', 'three'])

/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [117]:
#同样的逻辑也适用于 DataFrame 对象:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a',
'b', 'c'],
                   columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [118]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2),
index=['a', 'c'],
                   columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [120]:
pd.concat([df1, df2], axis=1)

/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [119]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [121]:
#如果传入的不是列表而是一个字典，则字典的键就会被当做 keys 选项的值:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [122]:
#两个用于管理层次化索引创建方式的参数(参见表 8-3)。举个例 子，我们可以用 names 参数命名创建的轴级别:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

/Users/xl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [123]:
#最后一个关于 DataFrame 的问题是，DataFrame 的行索引不包含任何相关数 据:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a',
'b', 'c', 'd'])
df1

,a,b,c,d
0,-0.345694,1.179416,0.694688,-1.157737
1,-1.936062,1.059977,0.708869,-0.151159
2,0.344957,-0.715508,-1.212946,1.072656


In [124]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b',
'd', 'a'])
df2

,b,d,a
0,-0.549306,-2.612927,-0.691860
1,0.995265,-0.516190,-0.363487


In [126]:
#在这种情况下，传入 ignore_index=True 即可:
pd.concat([df1, df2], ignore_index=True, sort=False)

,a,b,c,d
0,-0.345694,1.179416,0.694688,-1.157737
1,-1.936062,1.059977,0.708869,-0.151159
2,0.344957,-0.715508,-1.212946,1.072656
3,-0.691860,-0.549306,NaN,-2.612927
4,-0.363487,0.995265,NaN,-0.516190


## 合并重叠数据

In [ ]:
#还有一种数据组合问题不能用简单的合并(merge)或连接(concatenation) 运算来处理。

In [128]:
#我们使用 NumPy 的 where 函数，它表示一种等价于面向数组的 if-else:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [129]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [132]:
b[-1] = np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [134]:
pd.isnull(a)

f     True
e    False
d     True
c    False
b    False
a     True
dtype: bool

In [133]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [136]:
#Series 有一个 combine_first 方法，实现的也是一样的功能，还带有 pandas 的数据对齐:

In [137]:
b[:-2]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64

In [138]:
a[2:]

d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [139]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [ ]:
#对于 DataFrame，combine_first 自然也会在列上做同样的事情，因此你可以将 其看做:用传递对象中的数据为调用对象的缺失数据“打补丁”:

In [140]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [141]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [142]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 重塑和轴向旋转

In [143]:
#有许多用于重新排列表格型数据的基础运算。这些函数也称作重塑(reshape) 或轴向旋转(pivot)运算。

## 重塑层次化索引

In [ ]:
#层次化索引为 DataFrame 数据的重排任务提供了一种具有良好一致性的方式。 主要功能有二:
1.stack:将数据的列“旋转”为行。
2.unstack:将数据的行“旋转”为列。

In [145]:
#简单的 DataFrame，其 中的行列索引均为字符串数组:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio','Colorado'],
ame='state'),
                    columns=pd.Index(['one', 'two',
'three'],
                    name='number'))
data

number,one,two,three
Ohio,0,1,2
Colorado,3,4,5


In [146]:
#对该数据使用 stack 方法即可将列转换为行，得到一个 Series:
result = data.stack()
result

          number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [147]:
#对于一个层次化索引的 Series，你可以用 unstack 将其重排为一个 DataFrame:
result.unstack()

number,one,two,three
Ohio,0,1,2
Colorado,3,4,5


In [150]:
#默认情况下，unstack 操作的是最内层(stack 也是如此)。传入分层级别的编 号或名称即可对其它级别进行 unstack 操作:
result.unstack(0)      # (未看懂todo)

,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


## 将“长格式”旋转为“宽格式”

## 将“宽格式”旋转为“长格式”

## 8.4 总结